In [1]:
import requests
from bs4 import BeautifulSoup
import time,re
import random , logstash , logging , pymongo



In [ ]:


host = '10.120.14.105'
client = "mongodb://" + host + ":27017/"
# 多執行續下 可能被 dead lock 
myclient = pymongo.MongoClient(client, connect=False)
mydb = myclient["mydatabase"]
mycol = mydb["Mobile01"]


In [ ]:
s = "我們都很好Pleas ======="



In [ ]:
re.sub(r"\W+","",s)


In [ ]:
# pipeline 
logger = logging.getLogger('python-logstash-logger')
logger.setLevel(logging.INFO)
host_demo = "logstash"
logger.addHandler(logstash.TCPLogstashHandler(host_demo,5959))




In [6]:
url = "https://www.mobile01.com/waypointlist.php?list=1&c=0&s=desc&p=1"

def get_url(n=1,m=3):
    result = []
    # 增加直覺 從第n頁 到 第m頁
    m = m + 1
    for t in range(n,m):
        url = "https://www.mobile01.com/waypointlist.php?list=1&c=0&s=desc&pid=2&p=" + str(t)
        result.append(url)
    return result
    

In [12]:
get_connect("https://www.mobile01.com/waypointlist.php?list=1&c=0&s=desc&pid=2&p=1")

<Response [200]>

In [11]:
proxylist = [{"http": "37.187.120.123:80"},
             {"https": "206.189.36.198:8080"},
             {"https": "178.128.31.153:8080"},
             {"http": "167.114.180.102:8080"},
             {"http": "104.131.214.218:80"},
             {"http": "167.114.196.153:80"}]
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

class Connect404Except(Exception):
    pass


# slove proxy connect error try next
def get_connect(url):
    i = random.randint(0,len(proxylist)-1)
    
    while True :
        
        try :
            resp= requests.get(url,headers=header,proxies=proxylist[i],timeout=120)          
            resp.encoding = "utf-8"
            time.sleep(random.randint(1,10)/10)
            #logger.info("Request response",exc_info=True,extra={"url" : url, "time_use" : resp.elapsed.total_seconds(), "Response code" : resp.status_code, "proxy_use":proxylist[i]})
           
            
        except requests.exceptions.ConnectionError :             
            #logger.warning("Proxy Error",exc_info=True,extra={"proxy":proxylist[i],"url":url})
            
            i = (i + 1) % len(proxylist)
            continue
        except requests.exceptions.ConnectTimeout :
            #logger.warning("Connect Time out",exc_info=True)
            i = (i + 1) % len(proxylist)
            continue

            
            
        
        if resp.status_code == requests.codes.ok :
            code = "HTTP response code = " + str(resp.status_code)
            
                
            break        
        if resp.status_code == 404 :
            #logger.warning("This page is gone nowhere !",exc_info=True,extra={"url":url})
                
            break
        i = i + 1
    
        
    return resp
    

In [37]:
# 餐廳資訊 評等
def get_res_info(soup):
    res_info = []
    info = soup.find(id="info_form")
    
    rate = len(info.find_all("div")[8].find_all("img"))
    place = info.find_all("div")[1].text.replace("\n","").replace(" ","").split(":")[-1]
    phone = info.find_all("div")[2].text.replace("\n","").replace(" ","").split(":")[-1]
    op_time = info.find_all("div")[3].text.replace("\n","").split("  ")[-1]
    point = info.find_all("div")[7].text.replace("\n","").replace(" ","").split(":")[-1]
    res_info.append({
        "rate" : rate,
        "place" : place,
        "phone" : phone,
        "op_time" : op_time,
        "point" : point
    })
    return res_info



In [43]:
def get_comment(soup):
    comment = []
    
    
    for single_comment in soup.find_all("article")[1:]:
        removes = single_comment.find_all("span", class_="poster")
        for remove in removes:
            remove.extract()
        removes = single_comment.find_all("blockquote")
        for remove in removes:
            remove.extract()
        author = single_comment.find("div", class_="single-post-author group").find("div", class_="fn").text
        content = single_comment.find("div", class_="single-post-content").text.replace("\n","").replace("\r","").replace(" ","")
        content = author + ":" + content
        comment.append(content)
        while True :
            the_link = ""
            links = soup.find_all("div",class_="pagination")[1].find_all("a")        
            for link in links :
                if link.find(string=re.compile("下一頁")):
                    the_link = "https://www.mobile01.com/"+link["href"]   
                    
            if the_link :
                # establish new connection
                conn = get_connect(the_link)
                try : 
                    if conn.status_code == 404 :
                        raise Connect404Except
                    else :
                        soup = BeautifulSoup(conn.text)
                except Connect404Except:                    
                    #logger.error("page error",extra={"url":the_link})
                    break
                # get our comment text
                if soup:
                    new_comment = get_comment_p2(soup)
                # append to comment list
                if new_comment:
                    for every_comment in new_comment:
                        comment.append(every_comment)
            else :
                break
    
    return comment


In [24]:
def get_comment_p2(soup):
    comment = []
    for single_comment in soup.find_all("article"):
        removes = single_comment.find_all("span", class_="poster")
        for remove in removes:
            remove.extract()
        removes = single_comment.find_all("blockquote")
        for remove in removes:
            remove.extract()
        author = single_comment.find("div", class_="single-post-author group").find("div", class_="fn").text
        content = single_comment.find("div", class_="single-post-content").text.replace("\n","").replace("\r","").replace(" ","")
        content = author + ":" + content
        comment.append(content)
    
    return comment

In [40]:
def parse(urls):
    result = []
    
    connect = get_connect(urls)
    try :
        if connect.status_code == 404:
            raise Connect404Except
            
        else :
            soup = BeautifulSoup(connect.text)
            
    except Connect404Except : 
        logger.error("index error",extra={"index":urls})
        pass
    
    if soup :
        print("進入",urls)
        table = soup.find_all("tr")
        # 取得列表內容
        for t in table:
            title = t.find("p", class_="title").text
            href = "https://www.mobile01.com/" + t.find("p", class_="title").find("a")["href"]
            sp = t.find("p", class_="info").text.split("-")[1].replace(" ","")
            date = t.find("p", class_="info").text.split(":")[2].split(" ")[1]
            
            # 進入第二層 遊記 去除風景
            if "餐飲" in sp:
                print("現在處理:",href)
                conn = get_connect(href)
                try :
                    
                    if conn.status_code == 404:
                        raise Connect404Except

                    else :
                        art = BeautifulSoup(conn.text)

                except Connect404Except : 
                    logger.error("page error",extra={"url":href})
                    continue
                content = art.find("div", class_="single-post-content").text.replace("\r","").replace("\n","").replace(" ","")
                content = re.sub(r"\W+","",content)
                img_url = []
                imgs = art.find("div", class_="single-post-content").find_all("img")
                for img in imgs:
                    img_url.append(img["data-src"])
                author = art.find("div", class_="panel note sidebar-authur").find("a").text
                article_hit = art.find_all("div",class_="panel note")[1].find_all("li")[0].text.split(":")[-1]
                rate = art.find_all("div",class_="panel note")[1].find_all("li")[1].text.split(":")[-1]
                res_info = get_res_info(art)
                # 進入第三層 評論
                comment_url = "https://www.mobile01.com/" + art.find("div", class_="btns wide").find("a")["href"]
                new_conn = get_connect(comment_url)
                try:
                    if new_conn.status_code == 404:
                        raise Connect404Except
                    else:
                        comment_art = BeautifulSoup(new_conn.text)
                except Connect404Except :
                    # logger.error("page error",extra={"url":comment_url})
                    pass
                if comment_art:
                    comment = get_comment(comment_art)
                
                # append all
                result.append({
                    "title" : title,
                    "href" : href,
                    "species" : sp,
                    "date" : date,
                    "content" : content,
                    "img" : img_url,
                    "author" : author,
                    "article_hit" : article_hit,
                    "rate" : rate,
                    "info" : res_info,
                    "comment" : comment
                    
                    
                })
                
    return result
                
                
                    
                
                    
                        
        
    



In [44]:
parse("https://www.mobile01.com/waypointlist.php?list=1&c=0&s=desc&pid=2&p=1")

進入 https://www.mobile01.com/waypointlist.php?list=1&c=0&s=desc&pid=2&p=1
現在處理: https://www.mobile01.com/waypointdetail.php?id=32603
現在處理: https://www.mobile01.com/waypointdetail.php?id=32597
現在處理: https://www.mobile01.com/waypointdetail.php?id=32526
現在處理: https://www.mobile01.com/waypointdetail.php?id=32522
現在處理: https://www.mobile01.com/waypointdetail.php?id=32490
現在處理: https://www.mobile01.com/waypointdetail.php?id=32487
現在處理: https://www.mobile01.com/waypointdetail.php?id=32476


[{'title': '▋捷運中山站美食 ▋First One Bistro 佛斯特旺餐酒館，超精彩的義式餐點秀！！',
  'href': 'https://www.mobile01.com/waypointdetail.php?id=32603',
  'species': '餐飲',
  'date': '2019-01-25',
  'content': '生活在台灣的好處就是以合理的價格嚐遍美食無論是東西方各式菜色都能在小小的地方得到味蕾的滿足這次我們到了台北捷運南西站附近的小巷內體驗到了驚豔的義大利菜色這一區其實我們很少來隨著google地圖繞著小巷找到餐廳其實它就在捷運站出口五十公尺處只是我們開車來把事情弄得複雜了一點週間的中午旁邊的餐廳早已開始排隊了這間FirstOneBistro不疾不徐踏著自己的腳步準時十二點才開門義式料理在台灣早已不僅是義大利麵與Pizza店門口攤開的菜單選擇之多著實讓人嚇一跳就算不進來看看菜單也會很飽喔一進門的感覺用餐空間比想像中要小一點頂多坐三十多人算是小巧型的不過裝潢卻很用心明暗對比強烈屬於有個性卻又帶點小溫馨的感覺右半邊是倣大理石桌使用的也是鐵椅靠左的是木質桌椅小小創意卻帶來不同的用餐氛圍還好一開門就進來拍等一下這邊就坐滿了客人菜單實在太豐富了還是請各位直接參考線上電子菜單法式燻鮭魚芥末籽沙拉蜂蜜麵包木盆的溫暖感覺滿點西西里炸飯糰我原本不知道這是什麼好奇點來吃看看好特別應該是將奶油燉飯裹粉後下去酥炸可是那個皮也太薄了吧這怎麼辦到的外面的酥皮薄而脆裡面的燉飯仍濕潤這個各位一定要吃看看前義式生火腿小點後煙燻鮭魚酸奶醬小點下面用的是比較生硬的法式麵包上面加了魚肉類別看它小小的吃起來頗有嚼勁喔我們是中午去的原本沒有想到要喝酒沒想到就是有人一直跟我使眼色說要喝啤酒不看也罷一看菜單不得了一堆沒看過的歐式啤酒可以選店長的建議下二位男士選了左邊的7的帝國黑啤王二位貴婦喝水果森林莓果啤酒左邊苦澀深沈右邊順口明亮齁齁配上接下來的義式下酒菜怎麼這麼搭西班牙香蒜油蝦好啦這是西班牙口味反正都是地中海周邊算同一國的來自挪威的燻鮭魚表示酥炸綜合海鮮我超想知道這間店的油鍋長什麼樣子怎麼弄出這麼薄的酥皮若拿這些魚蝦小卷去海產店弄出來的口感會完全不一樣馬札瑞拉紅醬培根義大利麵來義式餐廳沒吃麵就像去林東芳不吃牛肉麵一樣奇怪吧肉醬麵是最基本的這裡多出了培根的焦

In [25]:
art = BeautifulSoup(get_connect("https://www.mobile01.com/waypointtopicdetail.php?f=195&t=5694705#2").text)

In [27]:
links = art.find_all("div",class_="pagination")[1].find_all("a")
for link in links:
    if link.find(string=re.compile("下一頁")):
        the_link = link
    else :
        break


In [33]:
import re
# stackoverflow solution 
# https://stackoverflow.com/questions/31958637/beautifulsoup-search-by-text-inside-a-tag
art2 = BeautifulSoup(get_connect("https://www.mobile01.com/waypointtopicdetail.php?f=189&t=4462255").text)
links = art2.find_all("div",class_="pagination")[1].find_all("a")
for link in links :
    if link.find(string=re.compile("下一頁")):
        the_link = "https://www.mobile01.com/"+link["href"]
    
        
    

In [34]:
the_link

''

In [22]:
while the_link :
    print("a")

In [32]:
the_link = ""